<a href="https://colab.research.google.com/github/gusmaomarcos/machine-learning-avancada/blob/main/word2vec_classificador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!python -m spacy download pt_core_news_sm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import spacy
#import pt_core_news_sm

from gensim.models import KeyedVectors

In [ ]:
w2v_modelo_cbow = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Databases/word2vec_2/modelo_cbow.txt')
w2v_modelo_sg = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Databases/word2vec_2/modelo_skipgram.txt')

teste = "/content/drive/MyDrive/Databases/word2vec_2/teste.csv"
treino = "/content/drive/MyDrive/Databases/word2vec_2/treino.csv"

artigo_teste = pd.read_csv(teste)
artigo_treino = pd.read_csv(treino)

In [ ]:
nlp = spacy.load('pt_core_news_sm', disable=['parser', 'ner', 'tagger', 'textcat'])

def tokenizador(texto):
    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())

    return tokens_validos

texto = 'MERCEDES PROMETE MAIS ATUALIZAÇÕES PARA AS PRÓXIMAS CORRIDAS'
tokens = tokenizador(texto)
print(tokens)

['mercedes', 'promete', 'atualizações', 'próximas', 'corridas']


In [ ]:
def combinacao_de_vetores_por_soma(palavras, modelo):
    vetor_resultante = np.zeros(300)

    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)
        except KeyError:
            pass

    return vetor_resultante

vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
#print(vetor_texto)

[ 0.59530419 -1.07734841  0.36770131  0.15194771 -0.20767688  0.95430392
  0.88788593 -0.80822411 -0.92314301  0.65805054 -2.13131181 -0.78005192
  0.58454423  0.14748066  0.08369558 -1.03352138  2.42359552  0.15587878
 -0.221493    1.00122724 -1.75546175  0.43169922  1.20668842 -0.73070091
  2.05204287  0.34299631  0.61302301  0.08736861 -0.77551249  0.65940253
 -0.64854321 -0.55885904 -0.48426901  0.73180223 -0.62548091 -0.53340746
 -1.11058009 -0.10868419  1.70830433 -1.38651092 -2.14526627  2.13764572
  0.32960287 -1.67200828 -0.09541127  0.35700931 -0.36711256  1.17965975
 -0.47576071 -0.34742113 -1.08551154  0.34565646 -1.2931365  -0.99911633
 -0.60599634  1.40182677  0.55819917  0.41082618 -1.72160918  1.00999755
  0.17919517  2.96014974 -0.30954469  0.05653854 -0.885763   -0.20292845
 -0.02558921 -0.49619792  0.25520013 -1.13283639  0.28198233  1.79641049
 -0.88748146  1.69078062 -1.36754021 -1.04830903  0.17443937 -0.34147414
 -2.40850006  0.64497968  0.94879428 -1.25105568  0

In [ ]:
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras,modelo)

    return matriz

matriz_treino_cbow = matriz_vetores(artigo_treino['title'], w2v_modelo_cbow)
matriz_teste_cbow = matriz_vetores(artigo_teste['title'], w2v_modelo_cbow)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino, x_teste, y_teste):

    RL=LogisticRegression(max_iter=800)
    RL.fit(x_treino, y_treino)
    categorias = RL.predict(x_teste)
    print(classification_report(y_teste, categorias))
    return RL

RL_cbow = classificador(w2v_modelo_cbow,
                        matriz_treino_cbow, artigo_treino['category'],
                        matriz_teste_cbow, artigo_teste['category'])

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.63      0.80      0.70      1698
     esporte       0.93      0.86      0.89      4663
   ilustrada       0.13      0.85      0.23       131
     mercado       0.83      0.78      0.81      5867
       mundo       0.74      0.83      0.78      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [ ]:
matriz_treino_sg = matriz_vetores(artigo_treino['title'], w2v_modelo_sg)
matriz_teste_sg = matriz_vetores(artigo_teste['title'], w2v_modelo_sg)

RL_sg = classificador(w2v_modelo_sg,
                        matriz_treino_sg, artigo_treino['category'],
                        matriz_teste_sg, artigo_teste['category'])

              precision    recall  f1-score   support

     colunas       0.81      0.72      0.76      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.93      0.88      0.90      4663
   ilustrada       0.14      0.87      0.25       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.75      0.85      0.80      2051

    accuracy                           0.80     20513
   macro avg       0.69      0.82      0.71     20513
weighted avg       0.82      0.80      0.81     20513



In [ ]:
import pickle

with open('/content/drive/MyDrive/Databases/word2vec_2/rl_cbow.pkl', 'wb') as f:
    pickle.dump(RL_cbow, f)

In [ ]:
with open('/content/drive/MyDrive/Databases/word2vec_2/rl_sg.pkl', 'wb') as f:
    pickle.dump(RL_sg, f)

In [ ]:
#rerun